# 🏡 Previsão de Preço do Aluguel - Precificação de Aluguéis em Nova York  
        
        **Autor:** Luciano Arruda  
        **Data:** `2025-02-07`  
        **Objetivo:** Este notebook aplica um **modelo treinado de Machine Learning** para prever o preço do aluguel de um imóvel em Nova York.  

        ## 📌 Etapas do Processo  
        1. **Carregamento do modelo e do scaler** 🛠️  
        2. **Preparação dos dados de entrada** 🔄  
        3. **Aplicação do modelo para previsão do preço** 📉  
        4. **Conversão para BRL e EUR** 💰  
        5. **Conclusões e próximos passos** ✅  
        

In [3]:
# 📥 Importação das Bibliotecas
import os
import joblib
import numpy as np
import pandas as pd
import requests
import logging

# Configuração do logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Caminhos para o modelo e scaler treinados
model_path = "../models/random_forest.pkl"
scaler_path = "../models/scaler.pkl"
features_path = "../data/final/nyc_rental_data_features.csv"


## 🛠️ Carregamento do Modelo e do Scaler  

        O modelo foi previamente treinado utilizando **Random Forest Regressor** e os dados foram normalizados usando um **StandardScaler**.
        

In [4]:
# 📂 Carregando o Modelo e o Scaler
def carregar_modelo(caminho):
    """Carrega o modelo treinado a partir do arquivo especificado."""
    return joblib.load(caminho)

def carregar_scaler(caminho):
    """Carrega o scaler salvo a partir do arquivo especificado."""
    return joblib.load(caminho)

# Carregar modelo e scaler
modelo = carregar_modelo(model_path)
scaler = carregar_scaler(scaler_path)
print("✅ Modelo e Scaler carregados com sucesso!")


✅ Modelo e Scaler carregados com sucesso!


## 🔄 Preparação dos Dados de Entrada  

        Aplicamos a mesma **engenharia de atributos** utilizada no treinamento para preparar os dados antes da previsão.
        

In [5]:
# 🏡 Definição dos Dados do Apartamento para Previsão
apartamento = {
    'bairro_group': 'Manhattan',
    'latitude': 40.75362,
    'longitude': -73.98377,
    'room_type': 'Entire home/apt',
    'minimo_noites': 1,
    'numero_de_reviews': 45,
    'reviews_por_mes': 0.38,
    'calculado_host_listings_count': 2,
    'disponibilidade_365': 355
}

# Carregar colunas esperadas do conjunto de treinamento (exceto 'price_log')
df_features = pd.read_csv(features_path)
expected_columns = list(df_features.drop(columns=["price_log"]).columns)

# Criando DataFrame com os dados do apartamento
df_input = pd.DataFrame([apartamento])

# Criar novas features
df_input['densidade_imoveis'] = 1  # Valor padrão
df_input['proximidade_centro'] = np.sqrt((df_input["latitude"] - 40.7128) ** 2 + (df_input["longitude"] + 74.0060) ** 2)

# Codificação One-Hot
df_input = pd.get_dummies(df_input, columns=['bairro_group', 'room_type'], drop_first=True)

# Reindexar para garantir as mesmas colunas do conjunto de treino
df_input = df_input.reindex(columns=expected_columns, fill_value=0)

# Aplicação do scaler
df_input.loc[:, scaler.feature_names_in_] = scaler.transform(df_input[scaler.feature_names_in_])

print("✅ Dados de entrada preparados com sucesso!")


✅ Dados de entrada preparados com sucesso!


/tmp/ipykernel_336893/577779868.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.29903607]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_input.loc[:, scaler.feature_names_in_] = scaler.transform(df_input[scaler.feature_names_in_])
/tmp/ipykernel_336893/577779868.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.46467909]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_input.loc[:, scaler.feature_names_in_] = scaler.transform(df_input[scaler.feature_names_in_])
/tmp/ipykernel_336893/577779868.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.1496289]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_input.loc[:, scaler.fe

## 📉 Previsão do Preço do Aluguel  

        O modelo retorna o **log(price)**, então aplicamos a **exponencial inversa (`expm1`)** para obter o preço real.
        

In [6]:
# 🔍 Previsão do Preço
preco_log = modelo.predict(df_input)[0]
preco_sugerido = np.expm1(preco_log)

print(f"🏡 Preço sugerido para o imóvel: **${preco_sugerido:.2f}**")


🏡 Preço sugerido para o imóvel: **$207.08**


## 💰 Conversão para BRL e EUR  

        Utilizamos a API **AwesomeAPI** para converter o preço de USD para **Reais (BRL)** e **Euros (EUR)**.
        

In [7]:
# 💱 Função para Conversão de Moeda
def converter_moedas(valor_usd):
    """Converte o valor de USD para BRL e EUR usando a API AwesomeAPI."""
    url = "https://economia.awesomeapi.com.br/json/last/USD-BRL,USD-EUR"
    response = requests.get(url)
    data = response.json()

    usd_brl = float(data['USDBRL']['bid'])
    usd_eur = float(data['USDEUR']['bid'])

    valor_brl = valor_usd * usd_brl
    valor_eur = valor_usd * usd_eur

    print(f"🔹 == Conversão de Moedas == 🔹")
    print(f"💵 Valor em USD:	{valor_usd:.2f} $")
    print(f"🇧🇷💰 Valor em BRL:	{valor_brl:.2f} R$")
    print(f"💶 Valor em EUR:	{valor_eur:.2f} €")

# Aplicando a conversão
converter_moedas(preco_sugerido)


🔹 == Conversão de Moedas == 🔹
💵 Valor em USD:	207.08 $
🇧🇷💰 Valor em BRL:	1196.94 R$
💶 Valor em EUR:	200.58 €


## ✅ Conclusões e Insights

        📌 **Resumo:**  
        - O modelo previu um preço de aluguel baseado em atributos do imóvel.  
        